In [1]:
from dask.distributed import Client, LocalCluster
from dask import delayed

In [2]:
import dask.bag as db
import os
import json
from operator import itemgetter
from operator import add

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

from collections import Counter

import time

import numpy as np

In [ ]:
#make client

client = Client()
client

In [ ]:
#load data and repartition
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
js = lines.map(json.loads).repartition(10)

## definition of functions

In [ ]:
#merge all the body texts in one for each file
def merge(record):
    text=''
    for rec in record:
        text+=rec['text']
    return text

texts = js.pluck("body_text").map(merge)
texts.take(1)

In [ ]:
#function for text cleaning
#we remove punctuation, numbers and stopwords 
#stopwords are taken from a library but also definied by us
#after this files are lists of words

tokenizer = RegexpTokenizer(r'\w+')
no_words={"i", "as", "or", "it", "et", "also", "may"}

def clean_func(text):

    def merge_text(text,stop_words):
        new_sentence =''
        for w in text:
            if w.lower() not in stop_words and w.isalpha(): 
                new_sentence += w 
                new_sentence += " "
        return new_sentence

    stop_words = set(stopwords.words('english'))
    result = word_tokenize(text)
    result = merge_text(result,stop_words.union(no_words))
    result = tokenizer.tokenize(result)
    return result

text_clean = texts.map(clean_func)
text_clean.take(1)

In [ ]:
# we transform each list in a list of dictionaries 
#where the unique words and their frequency is stored

def count_words(text):
    counts = dict(zip(Counter(text).keys(), Counter(text).values()))
    wList = [{"word":x , "counts": y} for x,y in counts.items()]
    return wList

words = text_clean.map(count_words)
words.take(1)

In [ ]:
# at last we sum over all file using the foldby method
# we pass the bag to the foldby after flattening it 
#the methos accesse the dictiornarios gropyng them with the value associated to the key word
# and the sum the counts of each word

def incr_amount(tot, x):
    return tot+x['counts']


total_counts = words.flatten().foldby('word', binop=incr_amount, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()

total_counts

In [ ]:
# at last we order the obtained list

total_counts_ordered = sorted(total_counts, key=itemgetter(1), reverse=True)
total_counts_ordered

In [ ]:
#timing of the operation

start = time.time()
word_count = (lines.map(json.loads).repartition(10).pluck('body_text')
                   .map(merge).map(clean_func)
                   .map(count_words).flatten()
                   .foldby('word', binop=incr_amount, 
                        initial=0, combine=add, 
                        combine_initial=0).compute())
word_sorted = sorted(word_count, key=itemgetter(1), reverse=True)
end = time.time()

print("time: ", end - start)
print(word_sorted[:10])

In [ ]:
client.close()

In [ ]:
cluster = LocalCluster(n_workers=2)
client = Client(cluster)

In [ ]:
start = time.time()
word_count = (lines.map(json.loads).repartition(10).pluck('body_text')
                   .map(merge).map(clean_func)
                   .map(count_words).flatten()
                   .foldby('word', binop=incr_amount, 
                        initial=0, combine=add, 
                        combine_initial=0).compute())
word_sorted = sorted(word_count, key=itemgetter(1), reverse=True)
end = time.time()

print("time: ", end - start)
print(word_sorted[:10])

In [ ]:
def get_time(workers=4, partitions=10):
    myCluster = LocalCluster(n_workers=workers)
    client = Client(myCluster) #make client
    start = time.time() #strat taking time
    word_count = (lines.map(json.loads).repartition(partitions).pluck('body_text')
                       .map(merge).map(clean_func)
                       .map(count_words).flatten()
                       .foldby('word', binop=incr_amount, 
                            initial=0, combine=add, 
                            combine_initial=0).compute())
    word_sorted = sorted(word_count, key=itemgetter(1), reverse=True)
    end = time.time()
    client.close() #close client
    myCluster.close() #close cluster
    return end-start
    

In [ ]:
#for nw in range (3,5):
#    print("nw ", nw, " time ", get_time(nw))

In [ ]:
#works = [1,2,3,4,5,6,7,8]
#parts = [1,2,5,10,50, 100, 150]
works = [8]
parts = [8]
for w in works:
    for p in parts:
        print("nw: ", w, " Partitions ", p, " time: ", get_time(w,p))

In [ ]:
get_time(12, 12)

In [ ]:
get_time(24, 48)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bokeh.palettes as palette

In [ ]:
height = [word_sorted[i][1] for i in range(0,len(word_sorted))]
bars = [word_sorted[i][0] for i in range(0,len(word_sorted))]
y_pos = np.arange(len(bars))

fig, ax = plt.subplots(figsize=(10,8))
index = 30
# Create bars
ax.barh(y_pos[:index], height[:index],color=palette.inferno(index))
plt.yticks(y_pos[:index], bars[:index])
ax.grid(True, which="both", ls="-",color='0.93')
ax.set_axisbelow(True)

## parte 2

In [ ]:
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
js = lines.map(json.loads).repartition(10)

In [ ]:
cluster=LocalCluster(n_workers=4)
client= Client(cluster)

In [ ]:
js.pluck('metadata').pluck('authors').take(2)

In [ ]:
authors = js.pluck('metadata').pluck('authors')

In [ ]:
def flatten(record):
    uni=loc=lab=''
    if 'institution' in record['affiliation'].keys():
        uni=record['affiliation']['institution']
    else: uni="Unknown"
    
    if 'laboratory' in record['affiliation'].keys():
        lab=record['affiliation']['laboratory']
    else: lab="Unknown"
    
    if 'location' in record['affiliation'].keys():
        if 'country' in record['affiliation']['location'].keys():
            loc=record['affiliation']['location']['country']
        else: loc="Unknown"
    else: loc="Unknown"
    
    if uni=='': uni="Unknown"
    if lab=='': lab="Unknown"
    if loc=='': loc="Unknown"
    
    return {
        'name':       record['first'],
        'surname':    record['last'], 
        'University': uni,
        'Laboratory': lab,
        'Country':    loc,
    }

authors.flatten().map(flatten).take(12)

In [ ]:
auth_df = authors.flatten().map(flatten).to_dataframe()
auth_df.head()

In [ ]:
univs = auth_df.University.value_counts().nlargest(10).compute()
univs

In [ ]:
labos = auth_df.Laboratory.value_counts().nlargest(10).compute()
labos

In [ ]:
countries = auth_df.Country.value_counts().nlargest(10).compute()
countries

In [ ]:
def get_time_univs(workers=4, partitions=10):
    myCluster = LocalCluster(n_workers=workers)
    client = Client(myCluster) #make client
    start = time.time() #strat taking time
    auth_df = (lines.map(json.loads).repartition(10)
                    .pluck('metadata').pluck('authors')
                    .flatten().map(flatten)
                    .to_dataframe())
    univs = auth_df.University.value_counts().nlargest(10).compute()
    end = time.time()
    client.close() #close client
    myCluster.close() #close cluster
    return end-start

def get_time_countries(workers=4, partitions=10):
    myCluster = LocalCluster(n_workers=workers)
    client = Client(myCluster) #make client
    start = time.time() #strat taking time
    auth_df = (lines.map(json.loads).repartition(10)
                    .pluck('metadata').pluck('authors')
                    .flatten().map(flatten)
                    .to_dataframe())
    countries = auth_df.Country.value_counts().nlargest(10).compute()
    end = time.time()
    client.close() #close client
    myCluster.close() #close cluster
    return end-start

def get_N_univs(workers=4, partitions=10, N=10):
    myCluster = LocalCluster(n_workers=workers)
    client = Client(myCluster) #make client
    auth_df = (lines.map(json.loads).repartition(10)
                    .pluck('metadata').pluck('authors')
                    .flatten().map(flatten)
                    .to_dataframe())
    univs = auth_df.University.value_counts().nlargest(N).compute()
    client.close() #close client
    myCluster.close() #close cluster
    return univs
def get_N_countries(workers=4, partitions=10, N=10):
    myCluster = LocalCluster(n_workers=workers)
    client = Client(myCluster) #make client
    auth_df = (lines.map(json.loads).repartition(10)
                    .pluck('metadata').pluck('authors')
                    .flatten().map(flatten)
                    .to_dataframe())
    countries = auth_df.Country.value_counts().nlargest(N).compute()
    client.close() #close client
    myCluster.close() #close cluster
    return countries
    

In [ ]:
get_time_univs(10,10)

In [ ]:
get_time_countries(10,10)

In [ ]:
get_N_univs(10,10, 10)

In [ ]:
get_N_countries(10,10, 10)

# parte 3

In [3]:
cluster=LocalCluster(n_workers=4)
client= Client(cluster)
client


Client Scheduler: tcp://127.0.0.1:37987 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.80 GB


In [6]:
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
js = lines.map(json.loads).repartition(10)

In [7]:
metas = js.pluck(["paper_id", "metadata"])
m=metas.take(1)[0]

In [8]:
print (m[1]["title"])

PfSWIB, a potential chromatin regulator for var gene regulation and parasite development in Plasmodium falciparum


In [11]:
import fasttext
import fasttext.util
ft = fasttext.load_model('/home/alessandro/Downloads/cc.en.300.bin')
ft.get_dimension() 
fasttext.util.reduce_model(ft, 100)

AttributeError: 'str' object has no attribute 'get_input_matrix'

In [ ]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data
#model = load_vectors('/home/alessandro/Downloads/wiki-news-300d-1M.vec')
#to get the embedding of word ’hello’:
#model['hello']

model = fast

In [ ]:
model.load

In [ ]:
text = (m[1]["title"])
text_split = text.split()

text_embedded = []
for t in text_split:
    try:
        text_embedded.append(model[t])
    except:
        pass
    
#text_embedded =[ model[t] for t in text_split]
text_embedded
#text_split

In [ ]:
try: 
    t = np.array(model['PfSWIB'])
except:
    t= np.zeros(1)

t

In [ ]:
def embedd (text):
    text_split = text.split()
    text_embedded = []
    for t in text_split:
        try:
            text_embedded.append(model[t])
        except:
            pass
    return text_embedded

def reco_emb(reco):
    print(reco)
    return {
        "paper_id": reco['paper_id'],
        "title": embedd(reco['title'])
    }

def flatten(reco):
    text= reco[1]['title']
    #emb_text=embedd(text)
    print(text)
    return {
        "paper_id": reco[0],
        "title": reco[1]['title']
    }

In [ ]:
titles = db.from_sequence(metas.map(flatten).compute())
titles.take(3)


#titles = metas.map(flatten_embedding)
#titles.take(3)

In [ ]:
t = titles.pluck('title').take(1)
#reco_emb(t[0])
t=t[0]
t
#ids = titles.pluck('paper_id').take(3)
#et = [embedd(x) for x in t]

model['hello']


In [ ]:
et = t.map(reco_emb).compute()


In [ ]:
et

In [ ]:
titles = metas.map(flatten).to_dataframe().compute())


In [ ]:
import itertools

#c = list(itertools.product(a, b))

In [ ]:
grid = db.from_sequence(list(itertools.product(small_titl, small_titl))).repartition(10)

In [ ]:
grid.take(2, npartitions=5)

In [ ]:
list(itertools.product(small_titl, small_titl))